# Freesurfer Segmentation and Volume Calculation

## Installation
Download and install freesurfer-Linux-centos6_x86_64-stable-pub-v6.0.0 from [here](https://surfer.nmr.mgh.harvard.edu/fswiki/rel6downloads).

## Usage
1. Preprocess MRI data in NIfTI format.
2. save them in the directory "sub" in accordance with the Brain Image Data Structure (BIDS)


In [ ]:
%%bash
#!/bin/bash
#(the segmentations is to be saved in the same dir. generated files end with .reconall)
export FREESURFER_HOME=$HOME/freesurfer
source $FREESURFER_HOME/SetUpFreeSurfer.sh

subs_dir="$HOME/sub"
export SUBJECTS_DIR=$subs_dir

cd $subs_dir
subs=($( ls -1d sub-* ))

echo ${subs[@]}

N=4

for s in ${subs[@]}
do
    ((i=i%N)); ((i++==0)) && wait
    if [ ! -d ${s}.reconall ] ; then
    	cd ${s} && \
    	files=($( ls -1 -r *.nii.gz)) && \
    	cd .. && \
        mri_convert ./${s}/$files ./${s}/orig.mgz && \
        recon-all -subjid $s.reconall -i ${s}/orig.mgz -all -parallel -openmp 4 &
    else
	echo "$s.reconall found. Skipping subject"
    fi
done



3. save the outputs of Freesurfer in dir FREESURFER_SEG


In [ ]:
%%bash
#calculate the vols
export FREESURFER_HOME=$HOME/freesurfer
source $FREESURFER_HOME/SetUpFreeSurfer.sh

curr_dir=$PWD
echo $PWD

base_dir=$(dirname "$curr_dir")
subs_dir="/FREESURFER_SEG" # change the path

cd $subs_dir
export SUBJECTS_DIR=$PWD
subs=($( ls -1d sub*))
N=20
for sub in "${subs[@]}"; do
((i=i%N)); ((i++==0)) && wait

    if [ ! -e $sub/stats/aparc.DKTatlas+aseg.fs.volume.stats ]; then
        mri_segstats --seg $sub/mri/aparc.DKTatlas+aseg.mgz \
                    --sum $sub/stats/aparc.DKTatlas+aseg.fs.volume.stats \
                    --pv $sub/mri/norm.mgz\
                    --empty\
                    --brainmask $sub/mri/brainmask.mgz\
                    --brain-vol-from-seg \
                    --excludeid 0 \
                    --subcortgray \
                    --in $sub/mri/norm.mgz \
                    --in-intensity-name norm \
                    --in-intensity-units MR --etiv \
                    --id 2, 4, 5, 7, 8, 10, 11, 12, 13, 14, 15, 16, 17, 18, 24, 26, 28, 31, 41, 43, 44, 46, \
                       47, 49, 50, 51, 52, 53, 54, 58, 60, 63, 77, 251, 252, 253, 254, 255, 1002, 1003, \
                       1005, 1006, 1007, 1008, 1009, 1010, 1011, 1012, 1013, 1014, 1015, 1016, 1017, 1018, \
                       1019, 1020, 1021, 1022, 1023, 1024, 1025, 1026, 1027, 1028, 1029, 1030, 1031, 1034, \
                       1035, 2002, 2003, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, \
                       2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024, 2025, 2026, 2027, 2028, 2029, \
                       2030, 2031, 2034, 2035 --ctab /$FREESURFER_HOME/FreeSurferColorLUT.txt \
                       --subject $sub &
    fi
done

exit

In [ ]:
%%bash
#add the vol values into a table
export FREESURFER_HOME=$HOME/freesurfer
source $FREESURFER_HOME/SetUpFreeSurfer.sh

curr_dir=$PWD
echo $PWD

base_dir=$(dirname "$curr_dir")
subs_dir="$/FREESURFER_SEG" # change the path
code_dir="$base_dir/code" # change the path


cd $subs_dir
subs=($( ls -1d sub*))
export SUBJECTS_DIR=$subs_dir
cd $code_dir
asegstats2table --subjects ${subs[@]} \
                --common-segs \
                --meas volume \
                --tablefile /freesurfer_aseg.volume.stats.csv \ #where to save the table (results)
                --statsfile=aparc.DKTatlas+aseg.fs.volume.stats \
                --skip 
                

exit

# Fastsurfer Segmentation and Volume Calculation

## Installation
Download and install freesurfer-Linux-centos6_x86_64-stable-pub-v6.0.0 from [here](https://surfer.nmr.mgh.harvard.edu/fswiki/rel6downloads).

## Usage
1. Preprocess MRI data in NIfTI format.
2. save them in the directory in accordance with the Brain Image Data Structure (BIDS) and refer to it in as src_dir


In [ ]:
%%bash

#!/bin/bash

## Install enviroment:
# conda create -n fastsurfer
# conda activate fastsurfer
# conda install pytorch torchvision cudatoolkit=11 cudnn -c pytorch
# conda install -c conda-forge numpy scipy scikit-image cycler decorator h5py imageio kiwisolver matplotlib nibabel Pillow pyparsing PyWavelets scikit-image six

#Fastsurfer needs freesurfer 6
export FREESURFER_HOME=$HOME/freesurfer
source $FREESURFER_HOME/SetUpFreeSurfer.sh

fastsurfer_dir="" #Dir with Fastsurfer cloned from Github
src_dir="" # Dir with subjects
dest_dir="" #Destination Dir (where the segmentations to be saved)
mkdir -p $dest_dir

cd $src_dir
subs=($(ls -1d sub-*))
echo ${subs[@]}
cd $fastsurfer_dir

for sub in ${subs[@]}
do
  echo $sub
  if [ ! -e "${dest_dir}/${sub}/mri/aparc.DKTatlas+aseg.deep.mgz" ]
  then
    t1=`ls ${src_dir}/${sub}/*.nii.gz`
    ./run_fastsurfer.sh --seg_only \
                        --vol_segstats \
                        --parallel --threads 20 \
                        --sd $dest_dir \
                        --sid $sub \
                        --t1 "$t1" \


  else
    echo "$sub seg found! skipping"
  fi
done

In [ ]:
%%bash
#add the vol values into a table

curr_dir=$PWD
echo $PWD

base_dir=$(dirname "$curr_dir")
subs_dir="$base_dir/derivatives/FASTSURFER_SEG"

cd $subs_dir
export SUBJECTS_DIR=$PWD
subs=($( ls -1d sub*))
asegstats2table --common-segs --meas volume \
                --skip --all-segs\
                --tablefile /fastsurfer_aseg.volume.stats.csv \ #where to save the table (results)
                --statsfile=aparc.DKTatlas+aseg.deep.volume.stats \
                --subjects "${subs[@]}"

at the end you get 2 tables fastsurfer_aseg.volume.stats.csv and freesurfer_aseg.volume.stats.csv